<a href="https://colab.research.google.com/github/sarabert96/Colexification/blob/main/01_DataUnderstanding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CLICS 3 Data Understanding

Import libraries

In [1]:
import pandas as pd
from google.colab import files
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

Obtaining raw dataframe from Google Drive

In [2]:
!gdown --id 1EySPG8ZDMfMTvhUQaw8w1k6NgywI4N4S #df_all_raw

Downloading...
From: https://drive.google.com/uc?id=1EySPG8ZDMfMTvhUQaw8w1k6NgywI4N4S
To: /content/df_all_raw.csv
244MB [00:01, 199MB/s]


In [3]:
df = pd.read_csv('df_all_raw.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
df.info

<bound method DataFrame.info of              dataset_ID  ... KUCERA_FRANCIS_FREQUENCY
0          abrahammonpa  ...                     52.0
1          abrahammonpa  ...                     52.0
2          abrahammonpa  ...                     52.0
3          abrahammonpa  ...                     52.0
4          abrahammonpa  ...                     52.0
...                 ...  ...                      ...
1390589  zgraggenmadang  ...                    242.0
1390590  zgraggenmadang  ...                    242.0
1390591  zgraggenmadang  ...                    242.0
1390592  zgraggenmadang  ...                    242.0
1390593  zgraggenmadang  ...                    242.0

[1390594 rows x 22 columns]>

In [10]:
df.head()

,dataset_ID,Form_ID,Form,clics_form,gloss_in_source,Concepticon_ID,Concepticon_Gloss,Ontological_Category,Semantic_Field,variety,Glottocode,ISO639P3code,Macroarea,Family,Latitude,Longitude,MRC_WORD,AGE_OF_ACQUISITION,CONCRETENESS,FAMILIARITY,IMAGABILITY,KUCERA_FRANCIS_FREQUENCY
0,abrahammonpa,BugunBichom-100_gold-1,san,san,gold,1369,GOLD,Person/Thing,Basic actions and technology,Bugun Bichom,bich1234,NaN,NaN,Sino-Tibetan,NaN,NaN,GOLD,NaN,576.0,550.0,594.0,52.0
1,abrahammonpa,BugunKaspi-100_gold-1,san,san,gold,1369,GOLD,Person/Thing,Basic actions and technology,Bugun Kaspi,kasp1234,NaN,NaN,Sino-Tibetan,NaN,NaN,GOLD,NaN,576.0,550.0,594.0,52.0
2,abrahammonpa,BugunNamphri-100_gold-1,san,san,gold,1369,GOLD,Person/Thing,Basic actions and technology,Bugun Namphri,namp1239,NaN,NaN,Sino-Tibetan,NaN,NaN,GOLD,NaN,576.0,550.0,594.0,52.0
3,abrahammonpa,BugunSingchung-100_gold-1,san,san,gold,1369,GOLD,Person/Thing,Basic actions and technology,Bugun Singchung,sing1271,NaN,NaN,Sino-Tibetan,NaN,NaN,GOLD,NaN,576.0,550.0,594.0,52.0
4,abrahammonpa,BugunWangho-100_gold-1,san,san,gold,1369,GOLD,Person/Thing,Basic actions and technology,Bugun Wangho,wang1301,NaN,NaN,Sino-Tibetan,NaN,NaN,GOLD,NaN,576.0,550.0,594.0,52.0


In [16]:
len(df.groupby('clics_form'))

706169

In [14]:
len(df.groupby('Concepticon_Gloss'))

2919

In [12]:
len(df.groupby('variety'))

3050

In [13]:
len(df.groupby('Glottocode'))

2279

In [15]:
len(df.groupby('Family'))

201

In [23]:
df.isna().sum()

dataset_ID                        0
Form_ID                           0
Form                             87
clics_form                      441
gloss_in_source               64676
Concepticon_ID                    0
Concepticon_Gloss                 0
Ontological_Category              0
Semantic_Field                  108
variety                           0
Glottocode                    19627
ISO639P3code                 206325
Macroarea                     87845
Family                        22090
Latitude                     226992
Longitude                    226992
MRC_WORD                     220322
AGE_OF_ACQUISITION          1060434
CONCRETENESS                 467034
FAMILIARITY                  423700
IMAGABILITY                  431767
KUCERA_FRANCIS_FREQUENCY     232552
dtype: int64

Adding tokenized column for a deeper analysis

In [4]:
df['tk_variety']=df['variety'].apply(nltk.tokenize.word_tokenize)


As soon as we find a diachronic variety, this is printed and the for stopped

In [16]:
for i in range(len(df['tk_variety'])):
  r = df['tk_variety'].iloc[i]
  if any((v == "Modern") for v in r): # change key word
    print (r)
    break

['Modern', 'Armenian']


Count how many specific varieties contains...

In [23]:
c = 0
for i in range(len(df['tk_variety'])):
  r = df['tk_variety'].iloc[i]
  if any((v == "Modern") for v in r): # change key word
    c+=1
print (c)

4256


Check for duplicated rows

In [30]:
df.loc[df.astype(str).drop_duplicates().index]

,dataset_ID,Form_ID,Form,clics_form,gloss_in_source,Concepticon_ID,Concepticon_Gloss,Ontological_Category,Semantic_Field,variety,Glottocode,ISO639P3code,Macroarea,Family,Latitude,Longitude,MRC_WORD,AGE_OF_ACQUISITION,CONCRETENESS,FAMILIARITY,IMAGABILITY,KUCERA_FRANCIS_FREQUENCY,tk_variety,flag
0,abrahammonpa,BugunBichom-100_gold-1,san,san,gold,1369,GOLD,Person/Thing,Basic actions and technology,Bugun Bichom,bich1234,NaN,NaN,Sino-Tibetan,NaN,NaN,GOLD,NaN,576.0,550.0,594.0,52.0,"[Bugun, Bichom]",0
1,abrahammonpa,BugunKaspi-100_gold-1,san,san,gold,1369,GOLD,Person/Thing,Basic actions and technology,Bugun Kaspi,kasp1234,NaN,NaN,Sino-Tibetan,NaN,NaN,GOLD,NaN,576.0,550.0,594.0,52.0,"[Bugun, Kaspi]",0
2,abrahammonpa,BugunNamphri-100_gold-1,san,san,gold,1369,GOLD,Person/Thing,Basic actions and technology,Bugun Namphri,namp1239,NaN,NaN,Sino-Tibetan,NaN,NaN,GOLD,NaN,576.0,550.0,594.0,52.0,"[Bugun, Namphri]",0
3,abrahammonpa,BugunSingchung-100_gold-1,san,san,gold,1369,GOLD,Person/Thing,Basic actions and technology,Bugun Singchung,sing1271,NaN,NaN,Sino-Tibetan,NaN,NaN,GOLD,NaN,576.0,550.0,594.0,52.0,"[Bugun, Singchung]",0
4,abrahammonpa,BugunWangho-100_gold-1,san,san,gold,1369,GOLD,Person/Thing,Basic actions and technology,Bugun Wangho,wang1301,NaN,NaN,Sino-Tibetan,NaN,NaN,GOLD,NaN,576.0,550.0,594.0,52.0,"[Bugun, Wangho]",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1390589,zgraggenmadang,yaben-9_boy-1,waw moya,wawmoya,boy,1366,BOY,Person/Thing,Kinship,yaben,yabe1255,ybm,Papunesia,Nuclear Trans New Guinea,-4.90868,145.361,BOY,NaN,609.0,606.0,618.0,242.0,[yaben],0
1390590,zgraggenmadang,yangulam-9_boy-1,ilaŋgum,ilanggum,boy,1366,BOY,Person/Thing,Kinship,yangulam,yang1298,ynl,Papunesia,Nuclear Trans New Guinea,-5.52697,145.858,BOY,NaN,609.0,606.0,618.0,242.0,[yangulam],0
1390591,zgraggenmadang,yangulam-9_boy-2,monaguru,monaguru,boy,1366,BOY,Person/Thing,Kinship,yangulam,yang1298,ynl,Papunesia,Nuclear Trans New Guinea,-5.52697,145.858,BOY,NaN,609.0,606.0,618.0,242.0,[yangulam],0
1390592,zgraggenmadang,yarawata-9_boy-1,wa tamana,watamana,boy,1366,BOY,Person/Thing,Kinship,yarawata,yara1250,yrw,Papunesia,Nuclear Trans New Guinea,-4.93943,145.454,BOY,NaN,609.0,606.0,618.0,242.0,[yarawata],0


Using a flag column, check concepticon with multiword, parentheses and OR

In [32]:
# create column with word count: flag > 0 if multiword
df['flag'] = df['Concepticon_Gloss'].str.count(' ')

In [34]:
symb=set('(-&')
for i in range(len(df['Concepticon_Gloss'])):
  r = df['Concepticon_Gloss'].iloc[i]
  if any((s in symb) for s in r):
    df['flag'].iloc[i]+=1

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [36]:
l=[]
for i in range(len(df['Concepticon_Gloss'])):
  r = df['Concepticon_Gloss'].iloc[i]
  n = df['flag'].iloc[i]
  if n > 0:
    l.append(r)

In [38]:
l=set(l)
l

{'A LITTLE',
 'ABSTAIN FROM FOOD',
 "ADAM'S APPLE",
 'AFFAIR (CLASSIFIER)',
 'ALCOHOL (FERMENTED DRINK)',
 'ALLOW OR PERMIT',
 'AMUSE ONESELF (HAVE FUN)',
 'ANACONDA (WATER BOA)',
 'ANGULAR (SQUAREFORMED)',
 'ANIMAL (CLASSIFIER)',
 'ANIMAL OR MEAT',
 'APRICOT BLOSSOM',
 'ARCTIC LIGHTS',
 'ARM OR HAND',
 'ASK (INQUIRE)',
 'ASK (REQUEST)',
 'AT FIRST',
 'AT THAT TIME',
 'AUTONYM (OF PEOPLE)',
 'AUTUMN OR HARVEST SEASON',
 'AXE (CLASSIFIER)',
 'BACK (DIRECTION)',
 'BACK (PART)',
 'BAD LUCK',
 'BALE-BALE (RAISED PLATFORM IN HOUSE)',
 'BAMBOO BASKET',
 'BAMBOO HAT',
 'BAMBOO MAT',
 'BAMBOO SHOOT',
 'BANANA TREE',
 'BAOBAB TREE',
 'BARK OR SKIN',
 'BARN OR SHED',
 'BATHE (SOMEONE)',
 'BATTLE-AXE',
 'BE ABLE',
 'BE ALIVE',
 'BE ALIVE OR LIFE',
 'BE ANGRY',
 'BE AT WAR',
 'BE BORN',
 'BE DEAD OR DIE',
 'BE DEFEATED',
 'BE HUNGRY',
 'BE IGNORANT',
 'BE LATE',
 'BE SILENT',
 'BE THIRSTY',
 'BEAN OR PEA',
 'BEAR FRUIT',
 'BEAT (CLASSIFIER)',
 'BECAUSE OF',
 'BECOME ANGRY',
 'BECOME EXTINGUISHED',

In [39]:
len(l)

908

In [ ]:
# eventually save in a txt file
#l.sort()
#with open('badConcepts.txt', 'w') as f:
#    for item in l:
#        f.write("%s\n" % item)